In [3]:
!pip install mediapipe opencv-python

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# 2. Draw Hands
<img src=https://i.imgur.com/qpRACer.png />

In [4]:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        
        #BGR to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Flip horizontal
        image = cv2.flip(image, 1)
        
        #Set flag
        image.flags.writeable = False
        
        #Detections
        results = hands.process(image)
        
        #Set flag back to True
        image.flags.writeable = True
        
        #RGB to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #print(results)
        
        #Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                         mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=4), 
                                         mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=4))

        #image = cv2.flip(image, 0)
        cv2.imshow("Hand Tracking", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    print(image.shape)

    cap.release()
    cv2.destroyAllWindows()

(480, 640, 3)


In [8]:
results.multi_hand_landmarks[0]

landmark {
  x: 0.7674967646598816
  y: 0.715493381023407
  z: 0.0
}
landmark {
  x: 0.7056344151496887
  y: 0.6646555662155151
  z: -0.021259723231196404
}
landmark {
  x: 0.6561721563339233
  y: 0.5901217460632324
  z: -0.04196333885192871
}
landmark {
  x: 0.6193229556083679
  y: 0.5278843641281128
  z: -0.0658707395195961
}
landmark {
  x: 0.5796563625335693
  y: 0.4834507405757904
  z: -0.0910552367568016
}
landmark {
  x: 0.7116186022758484
  y: 0.43809929490089417
  z: -0.022518789395689964
}
landmark {
  x: 0.6938372850418091
  y: 0.3248358964920044
  z: -0.05105402320623398
}
landmark {
  x: 0.6871813535690308
  y: 0.2531777024269104
  z: -0.07303804904222488
}
landmark {
  x: 0.6846256256103516
  y: 0.19048205018043518
  z: -0.08940228074789047
}
landmark {
  x: 0.764238178730011
  y: 0.4334039092063904
  z: -0.039719242602586746
}
landmark {
  x: 0.7705498337745667
  y: 0.30895882844924927
  z: -0.06552886962890625
}
landmark {
  x: 0.7781181931495667
  y: 0.2327151000499725

# Detect Left and Right Hand

In [1]:
results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.WRIST]

NameError: name 'results' is not defined

In [10]:
results.multi_handedness[0].classification[0]

index: 1
score: 0.9254442453384399
label: "Right"

In [3]:
def get_label(index, hand, results):
    output=None
    for idx, classification in enumerate(results.multi_handedness):
        if classification.classification[0].index == index:
            
            label = classification.classification[0].label
            score = classification.classification[0].score
            text = '{} {}'.format(label, round(score, 2))
            
            coords = tuple(np.multiply(
                np.array((hand.landmark[mp_hands.HandLandmark.THUMB_TIP].x, hand.landmark[mp_hands.HandLandmark.THUMB_TIP].y)),
                         [640, 480]).astype(int))
            
            output = text, coords
        
    return output

In [11]:
get_label(num, hand, results)

('Left 0.97', (210, 121))

In [18]:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        
        #BGR to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Flip horizontal
        image = cv2.flip(image, 1)
        
        #Set flag
        image.flags.writeable = False
        
        #Detections
        results = hands.process(image)
        
        #Set flag back to True
        image.flags.writeable = True
        
        #RGB to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #print(results)
        
        #Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                         mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=4), 
                                         mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=4))
                
                if get_label(num, hand, results):
                    text, coord = get_label(num, hand, results)
                    cv2.putText(image, text, coord, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        #image = cv2.flip(image, 0)
        cv2.imshow("Hand Tracking", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Line Between Fingers

In [23]:
x_index_finger = results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x
y_index_finger = results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP].y

x_thumb_finger = results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_TIP].x
y_thumb_finger = results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_TIP].y

0.4825209379196167

In [4]:
def get_position(landmark_idx = mp_hands.HandLandmark.INDEX_FINGER_TIP):
    x = int(results.multi_hand_landmarks[0].landmark[landmark_idx].x * 640)
    y = int(results.multi_hand_landmarks[0].landmark[landmark_idx].y * 480)
    z = results.multi_hand_landmarks[0].landmark[landmark_idx].y
    
    return x, y, z

def get_distance(x1, y1, x2, y2):
    distance = ((x1-x2)**2 + (y1-y2)**2)**0.5
    return distance
    

In [36]:
np.array([3, 3])

array([3, 3])

In [5]:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        
        #BGR to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Flip horizontal
        image = cv2.flip(image, 1)
        
        #Set flag
        image.flags.writeable = False
        
        #Detections
        results = hands.process(image)
        
        #Set flag back to True
        image.flags.writeable = True
        
        #RGB to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        

        #Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                         mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=4), 
                                         mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=4))
                
                
            x1, y1, z1 = get_position(mp_hands.HandLandmark.INDEX_FINGER_MCP)
            x2, y2, z2 = get_position(mp_hands.HandLandmark.WRIST)
            base = int(get_distance(x1, y1, x2, y2))
                
            x1, y1, z1 = get_position(mp_hands.HandLandmark.INDEX_FINGER_TIP)
            #cv2.putText(image, 'INDEX', (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            
            x2, y2, z2 = get_position(mp_hands.HandLandmark.THUMB_TIP)
            #cv2.putText(image, 'THUMB', (x2, y2), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            
            cv2.line(image, (x1, y1), (x2, y2), (255, 125, 100), 2)
            
            dist = str(int(100*get_distance(x1, y1, x2, y2)/base)) 
            cv2.putText(image, dist, (x2, y2), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            #cv2.putText(image, str(z1), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            
        #image = cv2.flip(image, 0)
        cv2.imshow("Hand Tracking", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()